In [127]:
from shared_functions import *

Load words from a text file and group them by word size.

In [128]:
word_list : list = load_words()

word_list_grouped_by_word_length : list[list] = group_words_by_length(word_list)
if (len(word_list) != sum(len(group) for group in word_list_grouped_by_word_length)) : raise AssertionError("word_list_grouped_by_word_length is missing some words.")

Calculate metrics about words
- That's a big number in terms of stack memory
- Test putting this in the program data segment for comparison

In [129]:
memSizeBytes = 0

for wordGroup in word_list_grouped_by_word_length:
    wordSize     = len(wordGroup[0])
    groupSize    = len(wordGroup)
    
    memSizeBytes += 1  * groupSize * wordSize

print(f"Word arrays' size: {memSizeBytes:,} bytes")

Word arrays' size: 3,494,695 bytes


Calculate metrics about Indexes
- That's a big number in terms of stack memory
- Test putting this in the program data segment for comparison

In [130]:
memSizeBytes = 0

for wordGroup in word_list_grouped_by_word_length:
    wordSize     = len(wordGroup[0])
    groupSize    = len(wordGroup)
    
    memSizeBytes += 2 * wordSize * groupSize

print(f"Index arrays' size: {memSizeBytes:,} bytes")

Index arrays' size: 6,989,390 bytes


Calculate metrics about Skip Indexes
- Hmm. That could be cut in half. Addition/subtraction is fast. The innermost array in the set only saves on an addition/subtraction opperation.
- Leave in stack, with the full-sized array for now. Lets see how much the number of page faults decreases with the changes listed above.

In [131]:
memSizeBytes = 0

for wordGroup in word_list_grouped_by_word_length:
    wordSize     = len(wordGroup[0])
    groupSize    = len(wordGroup)
    
    memSizeBytes += 2 * wordSize * 26 * 2

print(f"Skip index arrays' size: {memSizeBytes:,} bytes")

Skip index arrays' size: 45,760 bytes


Calculate metrics about wlPart
- ct      is uint16_t   : 2 bytes
- size    is uint8_t    : 1 bytes
- words   is char*      : 8 bytes (on x64)
- idxs    is uint16_t*  : 8 bytes (on x64)
- skIdxs  is uint16_t*  : 8 bytes (on x64)

In [132]:
wlPart_size_bytes = 2 + 1 + 8 + 8 + 8

print(f"Word List Parition structure size: {wlPart_size_bytes:,} bytes\n")
print(f"If I were to align this manually, I would be thinking about a 32-byte boundary\n") # ceil(n / 32) * 32
print(f"Since there are {len(word_list_grouped_by_word_length)} groups, these structures will take up ~{32 * len(word_list_grouped_by_word_length)} bytes in total.")

Word List Parition structure size: 27 bytes

If I were to align this manually, I would be thinking about a 32-byte boundary

Since there are 29 groups, these structures will take up ~928 bytes in total.


Calculate metrics about wordList and mutableWordList
- ct      is  uint16_t   : 2 bytes
- wlPart  is  wlPart[n]  : 32 * n
- sizes   is  uint8_t[n] : 8 * n

In [133]:
wordList_size_bytes = 2 + len(word_list_grouped_by_word_length) * (32 + 8)

print(f"Word List structure size: {wordList_size_bytes:,}\n")
print(f"If I were to align this manually, I would be thinking about a 1216-byte boundary\n") # ceil(n / 64) * 64

Word List structure size: 1,162

If I were to align this manually, I would be thinking about a 1216-byte boundary



Unrelated:


Ideas about how to store the search state
eg. last frame, the user typed in a character, this frame they pasted something, and now they hit backspace, etc..

I don't think you can do much better than a sorted array of ranges of indexes or memory locations.
Storing indexes would require less memory on 64-bit architectures.

struct {
    uint16_t start;
    uint16_t end;
} range; // 4 bytes

range* w1_results = malloc(1); // realloc ...
range* w2_results = malloc(1);

.
.
.

void* results[29]; // 232 bytes